In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import cufflinks as cf
import plotly.offline as plyo
import numpy as np
import numpy_financial as npf
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display
import stock_info_service 
from stock_info_service import *
from investment_rule import *
import importlib
cf.go_offline()
%matplotlib inline

In [2]:
df=cf.datagen.ohlc()
qf=cf.QuantFig(df,title='First Quant Figure',legend='top',name='GS')
qf.add_bollinger_bands()
plyo.iplot(qf.iplot())

PlotlyError: The `figure_or_data` positional argument must be `dict`-like, `list`-like, or an instance of plotly.graph_objs.Figure

In [4]:
    resp = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
    soup = BeautifulSoup(resp.text,'lxml')
    rows = [row for row in soup.find_all('tr') if row.findAll('td')[5]=='ESVUFR'  ]
    #rows = soup.find_all('tr',string='ESVUFR')
    stocks_info=[]
    tickers, securities, gics_industries, gics_sub_industries = [],[],[],[]
    for row in rows:
        ticker = row.findAll('td')[0].text
        security = row.findAll('td')[1].text
        gics_industry = row.findAll('td')[3].text
        gics_sub_industry = row.findAll('td')[4].text
        
        tickers.append(ticker.replace(r"\n"," "))
        securities.append(security)
        gics_industries.append(gics_industry)
        gics_sub_industries.append(gics_sub_industry)

    stocks_info.append(tickers)
    stocks_info.append(securities)
    stocks_info.append(gics_industries)
    stocks_info.append(gics_sub_industries)

    stocks_info_df = pd.DataFrame(stocks_info).T
    stocks_info_df.columns=['tickers','security','gics_industry','gics_sub_industry']
    stocks_info_df['seclabels'] = 'SP500'
    stocks_info_df['labels'] = stocks_info_df[['tickers','security', 'gics_industry','gics_sub_industry','seclabels']].apply(lambda x: ' '.join(x), axis=1)

KeyboardInterrupt: 

In [6]:
resp = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
soup = BeautifulSoup(resp.text,'lxml')
#rows = [row for row in soup.find_all('tr') if row.findAll('td')[5]=='ESVUFR'  ]

In [16]:
rows = [row for row in soup.find_all('tr') if len(row.findAll('td'))>5 and row.findAll('td')[5].text=='ESVUFR'  ]

In [27]:
    #resp = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
    #soup = BeautifulSoup(resp.text,'lxml')
    url = "./data/taiwan_stock_list_20200906.html"
    page = open(url,encoding='ms950')
    soup = BeautifulSoup(page.read())
    rows = [row for row in soup.find_all('tr') if len(row.findAll('td'))>5 and row.findAll('td')[5].text=='ESVUFR'  ]
    #rows = soup.find_all('tr',string='ESVUFR')
    stocks_info=[]
    tickers, securities, gics_industries, gics_sub_industries = [],[],[],[]
    for row in rows:
        company = row.findAll('td')[0].text.split()
        ticker = company[0]
        security = company[1]
        gics_industry = row.findAll('td')[3].text
        gics_sub_industry = row.findAll('td')[4].text
        
        tickers.append(ticker.replace(r"\n"," "))
        securities.append(security)
        gics_industries.append(gics_industry)
        gics_sub_industries.append(gics_sub_industry)

    stocks_info.append(tickers)
    stocks_info.append(securities)
    stocks_info.append(gics_industries)
    stocks_info.append(gics_sub_industries)

    stocks_info_df = pd.DataFrame(stocks_info).T
    stocks_info_df.columns=['tickers','security','gics_industry','gics_sub_industry']
    stocks_info_df['seclabels'] = 'SP500'
    stocks_info_df['labels'] = stocks_info_df[['tickers','security', 'gics_industry','gics_sub_industry','seclabels']].apply(lambda x: ' '.join(x), axis=1)


In [29]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')